In [68]:
import MeCab
from wordcloud import WordCloud
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import json

In [69]:
json_load = json.load(open('settings.json', 'r'))
CHASEN = json_load["CHASEN"]
IMG_PATH = "img/"

In [70]:
# tweetデータの読み込み
TWEET_DATA_PATH = "data/all_tweets.csv"
df = pd.read_csv(TWEET_DATA_PATH)
df["date"] = pd.to_datetime(df["date"])

In [73]:
m = MeCab.Tagger (r"-d '{}' -O chasen".format(CHASEN))

# 一番ふるい日付を取得。年と月
oldedst_date = min(df["date"])

# 期間開始日と終了日を生成
start_date = datetime.datetime(oldedst_date.year, oldedst_date.month, 1)
end_date = start_date + relativedelta(months=1, days=-1)

# 本日から見た翌月初日を生成
now = datetime.datetime.now()
limit_date = datetime.datetime(now.year, now.month, 1) + relativedelta(months=1)

cnt = 0
while start_date < limit_date:  # 今日からみた翌月になったらループ終了
    # 当該期間のデータ抽出
    tweets = df[(df["date"]>=start_date)&(df["date"]<=end_date)]["tweet_text"]

    # wordにわかちがいた名詞をスペース区切りで格納
    word=""
    for tweet in tweets:
        node = m.parseToNode(tweet)
        while node:
            hinshi = node.feature.split(",")[0]
            #if hinshi in ["名詞","動詞","形容詞"]:
            if hinshi in ["名詞"]:
                origin = node.feature.split(",")[6]
                word = word + " " + origin
            node = node.next
    
    # wordcloudで可視化
    fpath = "./ipaexg.ttf"
    wordcloud = WordCloud(background_color="white",font_path=fpath, width=600,height=400,min_font_size=15)
    wordcloud.generate(word)
    
    # 画像保存
    file_name = IMG_PATH + str(cnt) + "_" + str(start_date.year) + "-" + str(start_date.month) + ".png"
    wordcloud.to_file(file_name)
    
    # カーソル処理
    cnt += 1
    start_date += relativedelta(months=1)
    end_date = start_date + relativedelta(months=1, days=-1)
